# Multilayer Perceptron
**Comparing Federated Machine Learning to Centralized Machine Learning**

## Imports & Configs

In [1]:
from collections import OrderedDict
from typing import List, Tuple
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import multiprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets.utils.logging import disable_progress_bar
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

import flwr as fl
from flwr.client import Client, ClientApp, NumPyClient
from flwr.common import Metrics, Context
from flwr.server import ServerApp, ServerConfig, ServerAppComponents
from flwr.server.strategy import FedAvg
from flwr.simulation import run_simulation
from flwr_datasets import FederatedDataset

In [2]:
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    device_name = torch.cuda.get_device_name(DEVICE)
elif torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    device_name = "Apple Silicon"
else:
    DEVICE = torch.device("cpu")
    device_name = "CPU"
    
print(f"Training on {device_name}")
print(f"Flower {fl.__version__} / PyTorch {torch.__version__}")
disable_progress_bar()
n_cores = multiprocessing.cpu_count()

Training on Apple Silicon
Flower 1.14.0 / PyTorch 2.5.1


In [3]:
NUM_CLIENTS = 3
BATCH_SIZE = 32
NUM_EPOCHS = 1
NUM_ROUNDS = 3

torch.manual_seed(0)

## Loading Data

In [4]:
train_df = pd.read_csv('./Data/adult_train.csv')
test_df = pd.read_csv('./Data/adult_train.csv')
concated_df = pd.concat([train_df, test_df], ignore_index=True)

label_column_name = 'income'  
features = concated_df.drop(columns=[label_column_name]).values
labels = concated_df[label_column_name].values

In [5]:
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [6]:
def partition_dataset(features, labels, num_clients):
    dataset = CustomDataset(features, labels)
    dataset_size = len(dataset)
    partition_size = dataset_size // num_clients
    
    lengths = [partition_size] * num_clients
    lengths[-1] += dataset_size % num_clients
    partitions = random_split(dataset, lengths)
    return partitions

In [7]:
def load_federated_datasets(train_csv, test_csv, label_column_name, num_clients, batch_size):
    train_df = pd.read_csv(train_csv)
    train_features = train_df.drop(columns=[label_column_name]).values
    train_labels = train_df[label_column_name].values
    
    test_df = pd.read_csv(test_csv)
    test_features = test_df.drop(columns=[label_column_name]).values
    test_labels = test_df[label_column_name].values

    train_partitions = partition_dataset(train_features, train_labels, num_clients)
    test_partitions = partition_dataset(test_features, test_labels, num_clients)
    
    federated_trainloaders = []
    federated_testloaders = []

    for train_partition, test_partition in zip(train_partitions, test_partitions):
        trainloader = DataLoader(train_partition, batch_size=batch_size, shuffle=True)
        testloader = DataLoader(test_partition, batch_size=batch_size, shuffle=False)
        federated_trainloaders.append(trainloader)
        federated_testloaders.append(testloader)

    return federated_trainloaders, federated_testloaders

## Model

In [8]:
class Binary_MLP(nn.Module):
    def __init__(self):
        super(Binary_MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(97, 64),            
            nn.ReLU(),
            nn.Dropout(0.3),              
            nn.BatchNorm1d(64),            
            
            nn.Linear(64, 32),             
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.BatchNorm1d(32),
            
            nn.Linear(32, 16),             
            nn.ReLU(),
            nn.Dropout(0.2),
            
            nn.Linear(16, 1)               
        )
        
    def forward(self, x):
        return self.model(x)

In [9]:
def evaluate_model(model, testloader, device):
    model.eval()
    all_preds = []
    all_labels = []
    total_loss = 0.0
    criterion = nn.BCEWithLogitsLoss()  # Assuming binary classification with logits output

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device).float()  # Ensure labels are floats for BCELoss
            outputs = model(inputs)

            # Calculate loss for the batch
            batch_loss = criterion(outputs, labels.unsqueeze(1))  # Labels reshaped for compatibility
            total_loss += batch_loss.item()

            # Convert logits to predictions
            preds = torch.sigmoid(outputs).round()  # Threshold at 0.5 for binary classification
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)

    # Average loss over all batches
    avg_loss = total_loss / len(testloader)

    return avg_loss, accuracy, precision, recall, f1

## Centralized Training

In [10]:
centralized_dataset = CustomDataset(features, labels)
centralized_loader = DataLoader(centralized_dataset, batch_size=32, shuffle=True)

In [11]:
def centralized_training(model, loader, criterion, optimizer, num_epochs=NUM_EPOCHS):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        for inputs, labels in loader:
            inputs = inputs.to(DEVICE).float()
            labels = labels.to(DEVICE).float().unsqueeze(1)  # Ensure correct shape
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(loader)}")

In [12]:
model = Binary_MLP().to(DEVICE)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

centralized_training(model, centralized_loader, criterion, optimizer)

Epoch 1, Loss: 0.3609684840474124


In [13]:
c_loss, c_accuracy, c_precision, c_recall, c_f1 = evaluate_model(model, centralized_loader, DEVICE)

print(f"Centralized Model - Average Loss: {c_loss}, Accuracy: {c_accuracy}, Precision: {c_precision}, Recall: {c_recall}, F1: {c_f1}")

Centralized Model - Average Loss: 0.31206513738922026, Accuracy: 0.8553177113725009, Precision: 0.7667007498295841, Recall: 0.5737788547379161, F1: 0.6563571376468014


In [14]:
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

model.0.weight 	 torch.Size([64, 97])
model.0.bias 	 torch.Size([64])
model.3.weight 	 torch.Size([64])
model.3.bias 	 torch.Size([64])
model.3.running_mean 	 torch.Size([64])
model.3.running_var 	 torch.Size([64])
model.3.num_batches_tracked 	 torch.Size([])
model.4.weight 	 torch.Size([32, 64])
model.4.bias 	 torch.Size([32])
model.7.weight 	 torch.Size([32])
model.7.bias 	 torch.Size([32])
model.7.running_mean 	 torch.Size([32])
model.7.running_var 	 torch.Size([32])
model.7.num_batches_tracked 	 torch.Size([])
model.8.weight 	 torch.Size([16, 32])
model.8.bias 	 torch.Size([16])
model.11.weight 	 torch.Size([1, 16])
model.11.bias 	 torch.Size([1])


## Federated Training

In [15]:
backend_config = {
    "client_resources": {
        "num_cpus": n_cores,
        "num_gpus": 0
    }
}

In [16]:
def set_parameters(model, parameters):
    params_dict = zip(model.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.from_numpy(v) for k, v in params_dict})
    model.load_state_dict(state_dict, strict=True)

def get_parameters(model) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in model.state_dict().items()]

In [17]:
class FlowerClient(NumPyClient):
    def __init__(self, model, trainloader, valloader):
        self.model = model
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.model)

    def fit(self, parameters, config):
        set_parameters(self.model, parameters)
        centralized_training(self.model, centralized_loader, criterion, optimizer)
        return get_parameters(self.model), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.model, parameters)
        loss, accuracy, precision, recall, f1 = evaluate_model(self.model, self.valloader, DEVICE)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

In [18]:
def client_fn(context: Context) -> Client:
    model = Binary_MLP().to(DEVICE)

    partition_id = context.node_config["partition-id"]

    trainloaders, testloaders = load_federated_datasets(
        train_csv='./Data/adult_train.csv', 
        test_csv='./Data/adult_test.csv',   
        label_column_name='income',
        num_clients=NUM_CLIENTS,
        batch_size=BATCH_SIZE
    )

    trainloader = trainloaders[partition_id]
    testloader = testloaders[partition_id]

    return FlowerClient(model, trainloader, testloader).to_client()

In [19]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    return {"accuracy": sum(accuracies) / sum(examples)}

In [20]:
def server_fn(context: Context) -> ServerAppComponents:
    strategy = FedAvg(
        fraction_fit=1.0,  # Sample 100% of available clients for training
        fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
        min_fit_clients=10,  # Never sample less than 10 clients for training
        min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
        min_available_clients=10, # Wait until all 10 clients are available
        evaluate_metrics_aggregation_fn=weighted_average
    )
    config = ServerConfig(num_rounds=NUM_ROUNDS)
    return ServerAppComponents(strategy=strategy, config=config)


In [21]:
client = ClientApp(client_fn=client_fn)
server = ServerApp(server_fn=server_fn)

In [22]:
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config,
)

INFO :      Starting Flower ServerApp, config: num_rounds=5, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
INFO :      Received initial parameters from one random client
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


(ClientAppActor pid=19089) Epoch 1, Loss: 0.6897603144287361
(ClientAppActor pid=19089) Epoch 1, Loss: 0.6897036116460452
(ClientAppActor pid=19089) Epoch 1, Loss: 0.690087701430021
(ClientAppActor pid=19089) Epoch 1, Loss: 0.6896419559105909
(ClientAppActor pid=19089) Epoch 1, Loss: 0.6900033770176657
(ClientAppActor pid=19089) Epoch 1, Loss: 0.6903956571707323
(ClientAppActor pid=19089) Epoch 1, Loss: 0.6891804798588538
(ClientAppActor pid=19089) Epoch 1, Loss: 0.6904457883364091
(ClientAppActor pid=19089) Epoch 1, Loss: 0.6898109816263607


INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)


(ClientAppActor pid=19089) Epoch 1, Loss: 0.6900139429948653


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


...

INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 5]
INFO :      configure_fit: strategy sampled 10 clients (out of 10)


(ClientAppActor pid=19089) Epoch 1, Loss: 0.6903049175010917
(ClientAppActor pid=19089) Epoch 1, Loss: 0.690226950497899
(ClientAppActor pid=19089) Epoch 1, Loss: 0.6900586696059858
(ClientAppActor pid=19089) Epoch 1, Loss: 0.6901958337994127
(ClientAppActor pid=19089) Epoch 1, Loss: 0.6896055475726343
(ClientAppActor pid=19089) Epoch 1, Loss: 0.690113092423188
(ClientAppActor pid=19089) Epoch 1, Loss: 0.6900805088824983
(ClientAppActor pid=19089) Epoch 1, Loss: 0.6903381015088095
(ClientAppActor pid=19089) Epoch 1, Loss: 0.6902559034540986


INFO :      aggregate_fit: received 10 results and 0 failures
INFO :      configure_evaluate: strategy sampled 5 clients (out of 10)


(ClientAppActor pid=19089) Epoch 1, Loss: 0.689801474275664


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 5 round(s) in 417.08s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.6958953090742522
INFO :      		round 2: 0.6942171325870589
INFO :      		round 3: 0.6952295590849483
INFO :      		round 4: 0.6970425358005599
INFO :      		round 5: 0.6955551182522494
INFO :      	History (metrics, distributed, evaluate):
INFO :      	{'accuracy': [(1, 0.46167076167076165),
INFO :      	              (2, 0.478009828009828),
INFO :      	              (3, 0.4600737100737101),
INFO :      	              (4, 0.45768375105391684),
INFO :      	              (5, 0.4634577822423126)]}
INFO :      


## Comparison